In [1]:
from db_queries import get_population, get_ids
from db_queries import get_location_metadata as get_locs

In [2]:
import pandas as pd, numpy as np

# LSFF: choose population coverage data by hand for tier 4 countries

## vehicles: Wheat flour, maize flour, oil

## countries: Cameroon, Cote d'Ivoire, DRC, Indonesia, Mozambique

In [3]:
nutrients = ['iron','zinc','folic acid','vitamin a']

In [4]:
data_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/extraction_sheet_lsff_03_24_2021.3.csv'
assm_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/extraction_sheet_lsff_assumed_coverage_03_24_2021.csv'

df = pd.read_csv(data_path)

In [5]:
df.loc[df.nutrient=="folic acid, folate, b9",'nutrient']= 'folic acid'

In [6]:
assum = pd.read_csv(assm_path)

In [7]:
#these don't apply this time
assum.location_name.unique()

array(['Ethiopia', 'Myanmar', 'India'], dtype=object)

In [8]:
assum.loc[assum.nutrient=="folic acid, folate, b9",'nutrient']= 'folic acid'

In [9]:
df['estimation_status'] = 'na'

In [10]:
df['data_choice_notes'] = ""

In [11]:
mult_estimates_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/lsff_data_estimated_03_24_2021.csv'

mult_estimates = pd.read_csv(mult_estimates_path)

In [12]:
#reformat
mult_estimates.loc[(mult_estimates.B_estimate!=1.0),'B'] = np.nan
mult_estimates.loc[(mult_estimates.C_estimate!=1.0),'C'] = np.nan

mult_estimates = pd.melt(mult_estimates,
                         id_vars = ['location_name','vehicle','nutrient','standard'],
                         value_vars = ['B','C'], var_name = 'value_description', value_name = 'value_mean').dropna()

mult_estimates.loc[(mult_estimates.value_description=="B"),'nutrient'] = 'NA'

mult_estimates = mult_estimates.drop_duplicates()

mult_estimates.value_description = mult_estimates.value_description.map({
    'B':'percent of population eating industrially produced vehicle',
    'C':'percent of population eating fortified vehicle'
})

In [13]:
mult_estimates['estimation_status'] = 'multiplicative'

In [14]:
def prep_reg_estimates(path):
    draws = [f'draw_{i}' for i in range(500)]

    df = pd.read_csv(path)
    df = df.groupby(['location_name','vehicle']).mean().reset_index()
    df['value_mean'] = df[draws].mean(axis=1)
    df['value_025_percentile'] = df[draws].quantile(.025, axis=1)
    df['value_975_percentile'] = df[draws].quantile(.975, axis=1)

    return df[['location_name','vehicle','value_mean']]

In [15]:
output_dir = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/'

reg_fort_oil_path = output_dir + 'pct_eating_fortified_oil_regression_estimates_3_22_2021.csv'
reg_fort_wheat_path = output_dir + 'pct_eating_fortified_wheat_regression_estimates_3_22_2021.csv'
reg_fort_maize_path = output_dir + 'pct_eating_fortified_maize_regression_estimates_3_22_2021.csv'

fort_oil = prep_reg_estimates(reg_fort_oil_path)
fort_wheat = prep_reg_estimates(reg_fort_wheat_path)
fort_maize = prep_reg_estimates(reg_fort_maize_path)

In [18]:
## load legal combos
import pickle
data_prep_dir = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/'

with open(data_prep_dir + 'lsff_vehicle_nutrient_pairs.pickle', 'rb') as handle:
    vehicle_nutrient_map = pickle.load(handle)
    
with open(data_prep_dir + 'lsff_country_vehicle_pairs.pickle', 'rb') as handle:
    country_vehicle_map = pickle.load(handle)
    
with open(data_prep_dir + 'lsff_vehicle_country_pairs.pickle', 'rb') as handle:
    vehicle_country_map = pickle.load(handle)

In [19]:
location_names = ['Cameroon',"Côte d'Ivoire",'Democratic Republic of the Congo','Indonesia','Mozambique']
vehicles = ['maize flour','wheat flour','oil']
nutrients = ['folic acid','iron','zinc','vitamin a']

In [20]:
# these are the vehicles per country we need
target_high_level = pd.DataFrame([(loc,v) for loc in location_names for v in country_vehicle_map[loc]],
            columns=['location_name','vehicle']).sort_values(['location_name','vehicle'])

target_high_level = target_high_level[target_high_level.vehicle.isin(vehicles)].set_index(['location_name','vehicle'])

target_high_level

Empty DataFrame
Columns: []
Index: [(Cameroon, maize flour), (Cameroon, oil), (Cameroon, wheat flour), (Côte d'Ivoire, maize flour), (Côte d'Ivoire, oil), (Côte d'Ivoire, wheat flour), (Democratic Republic of the Congo, maize flour), (Democratic Republic of the Congo, oil), (Democratic Republic of the Congo, wheat flour), (Indonesia, oil), (Indonesia, wheat flour), (Mozambique, maize flour), (Mozambique, oil), (Mozambique, wheat flour)]

In [21]:
target_a = pd.DataFrame([(loc,vehicle,nutrient,'percent of population eating fortified vehicle') for loc in location_names
                       for vehicle in country_vehicle_map[loc]
                      for nutrient in vehicle_nutrient_map[vehicle]],
            columns=['location_name','vehicle','nutrient','value_description']).sort_values(['location_name','vehicle','nutrient'])
target_a = target_a[(target_a.nutrient.isin(nutrients))]

target_b = pd.DataFrame([(loc,vehicle,'na',val) for loc in location_names
                       for vehicle in country_vehicle_map[loc]
                      for val in ['percent of population eating industrially produced vehicle',
       'percent of population eating vehicle']],
            columns=['location_name','vehicle','nutrient','value_description'])

In [22]:
sortvars = ['location_name','vehicle','value_description','nutrient']
target = target_a.append(target_b)
target = target[(target.vehicle.isin(vehicles)) & (target.nutrient.isin(nutrients + ['na']))].sort_values(sortvars).set_index(sortvars)

In [23]:
target = target.reset_index()
rcols = target.columns.tolist()

In [24]:
check_cols = ['location_id','location_name','urbanicity','subnational_name','vehicle','value_description','nutrient','value_mean','value_025_percentile',
       'value_975_percentile','sub_population','source_year','notes','source_citation','source_link','inclusion_justification','included','data_choice_notes']

def filter_data(country, vehicle, val):    
    output = df.loc[(df.location_name==country)
           & (df.vehicle==vehicle)
           & (df.value_description==val)
           & (df.value_mean.notna()),check_cols]
    
    return output

In [25]:
def check_one_country(country):
    vehicles = ['oil', 'wheat flour', 'salt', 'maize flour', 'rice', 'bouillon']
    values_gold = ['percent of population eating fortified vehicle',
               'percent of population eating industrially produced vehicle',
               'percent of population eating vehicle']
    return pd.concat([filter_data(country, vehicle, val) for vehicle in vehicles for val in values_gold])

In [26]:
usecols = ['location_id','location_name','subnational_name','vehicle','value_description','nutrient','value_mean', 'value_025_percentile',
       'value_975_percentile']
subset_data = {}

In [27]:
for i in location_names:
    subset_data[i] = pd.DataFrame()

In [28]:
location_names

['Cameroon',
 "Côte d'Ivoire",
 'Democratic Republic of the Congo',
 'Indonesia',
 'Mozambique']

## Cameroon

In [29]:
cameroon = check_one_country("Cameroon")

#nathaniel marked these for inclusion
cameroon.data_choice_notes = cameroon.inclusion_justification

cameroon[(cameroon.vehicle.isin(vehicles))].groupby(['vehicle','value_description']).mean()

/ihme/code/central_comp/miniconda_svc-ccomp/envs/v107/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,,location_id
vehicle,value_description,
oil,percent of population eating vehicle,NaN
wheat flour,percent of population eating vehicle,NaN


In [30]:
mult_estimates[(mult_estimates.location_name=="Cameroon") & (mult_estimates.vehicle.isin(vehicles))]

,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status
52,Cameroon,wheat flour,NA,Mandatory,percent of population eating industrially prod...,75.570909,multiplicative
55,Cameroon,wheat flour,NA,Unknown,percent of population eating industrially prod...,75.570909,multiplicative
401,Cameroon,oil,vitamin a,Mandatory,percent of population eating fortified vehicle,30.496900,multiplicative
402,Cameroon,oil,vitamin d,Unknown,percent of population eating fortified vehicle,0.000000,multiplicative
403,Cameroon,wheat flour,folic acid,Mandatory,percent of population eating fortified vehicle,78.215891,multiplicative
404,Cameroon,wheat flour,iron,Mandatory,percent of population eating fortified vehicle,59.724634,multiplicative
406,Cameroon,wheat flour,vitamin b1,Unknown,percent of population eating fortified vehicle,0.000000,multiplicative
407,Cameroon,wheat flour,vitamin b12,Mandatory,percent of population eating fortified vehicle,78.215891,multiplicative
408,Cameroon,wheat flour,zinc,Mandatory,percent of population eating fortified vehicle,78.215891,multiplicative


In [31]:
subset_data['Cameroon'] = subset_data['Cameroon'].append(
    cameroon[(cameroon.vehicle=="oil") & (cameroon.included)]
)

cameroon[(cameroon.vehicle=="oil") & (cameroon.included)]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,inclusion_justification,included,data_choice_notes
610,NaN,Cameroon,mixed/both,na,oil,percent of population eating vehicle,na,53.1,na,na,under-5,2009,Under-5 defined as age 6-59 months. Proportion...,"Engle-Stone R, Ndjebayi AO, Nankap M, Brown KH...",https://watermark.silverchair.com/555.pdf?toke...,Keeping 53.1% for mixed/both urbanicity from E...,True,Keeping 53.1% for mixed/both urbanicity from E...


In [32]:
subset_data['Cameroon'] = subset_data['Cameroon'].append(
    mult_estimates[(mult_estimates.location_name=="Cameroon") 
               & (mult_estimates.vehicle=="oil") 
               & (mult_estimates.nutrient.isin(nutrients))]
)

mult_estimates[(mult_estimates.location_name=="Cameroon") 
               & (mult_estimates.vehicle=="oil") 
               & (mult_estimates.nutrient.isin(nutrients))]

/ihme/code/central_comp/miniconda_svc-ccomp/envs/v107/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status
401,Cameroon,oil,vitamin a,Mandatory,percent of population eating fortified vehicle,30.4969,multiplicative


In [33]:
## need pct eating ind prod oil from regression

In [34]:
subset_data['Cameroon'] = subset_data['Cameroon'].append(
    cameroon[(cameroon.vehicle=="wheat flour") & (cameroon.included)]
)

cameroon[(cameroon.vehicle=="wheat flour") & (cameroon.included)]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,inclusion_justification,included,data_choice_notes
575,NaN,Cameroon,mixed/both,na,wheat flour,percent of population eating vehicle,na,92,90,94,women of reproductive age,2011,WRA age defined as age 16-45. Numbers from Fo...,"Hess SY, Brown KH, Sablah M, Engle-Stone R, Aa...",https://journals.sagepub.com/doi/pdf/10.1177/1...,Keeping value from Hess et al. (92%) for mixed...,True,Keeping value from Hess et al. (92%) for mixed...
577,NaN,Cameroon,mixed/both,na,wheat flour,percent of population eating vehicle,na,94,92,96,under-5,2011,Under-5 defined as age 12-59 months. Numbers f...,"Hess SY, Brown KH, Sablah M, Engle-Stone R, Aa...",https://journals.sagepub.com/doi/pdf/10.1177/1...,Keeping value from Hess et al. (94%) from 2011...,True,Keeping value from Hess et al. (94%) from 2011...


In [35]:
subset_data['Cameroon'] = subset_data['Cameroon'].append(
    mult_estimates[(mult_estimates.location_name=="Cameroon") 
               & (mult_estimates.vehicle=="wheat flour") 
               & (mult_estimates.nutrient.isin(nutrients))]
)


mult_estimates[(mult_estimates.location_name=="Cameroon") 
               & (mult_estimates.vehicle=="wheat flour") 
               & (mult_estimates.nutrient.isin(nutrients))]

,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status
403,Cameroon,wheat flour,folic acid,Mandatory,percent of population eating fortified vehicle,78.215891,multiplicative
404,Cameroon,wheat flour,iron,Mandatory,percent of population eating fortified vehicle,59.724634,multiplicative
408,Cameroon,wheat flour,zinc,Mandatory,percent of population eating fortified vehicle,78.215891,multiplicative


In [36]:
## need pct eating ind prod wheat flour from regression

In [37]:
mult_estimates[(mult_estimates.location_name=="Cameroon") 
               & (mult_estimates.vehicle=="maize flour")]

,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status


In [38]:
## need pct eating ind prod wheat flour from regression

In [39]:
## need all maize flour vals from regression

In [40]:
vcols = ['location_name','vehicle','value_description','nutrient']
subset_data['Cameroon'].sort_values(vcols).set_index(vcols)

data_choice_notes  \
location_name vehicle     value_description                              nutrient                                                        
Cameroon      oil         percent of population eating fortified vehicle vitamin a                                                 NaN   
                          percent of population eating vehicle           na          Keeping 53.1% for mixed/both urbanicity from E...   
              wheat flour percent of population eating fortified vehicle folic acid                                                NaN   
                                                                         iron                                                      NaN   
                                                                         zinc                                                      NaN   
                          percent of population eating vehicle           na          Keeping value from Hess et al. (92%) for mixed...   
                                                                         na          Keeping value from Hess et al. (94%) from 2011...   

                                                                                    estimation_status  \
location_name vehicle     value_description                              nutrient                       
Cameroon      oil         percent of population eating fortified vehicle vitamin a     multiplicative   
                          percent of population eating vehicle           na                       NaN   
              wheat flour percent of population eating fortified vehicle folic acid    multiplicative   
                                                                         iron          multiplicative   
                                                                         zinc          multiplicative   
                          percent of population eating vehicle           na                       NaN   
                                                                         na                       NaN   

                                                                                    included  \
location_name vehicle     value_description                              nutrient              
Cameroon      oil         percent of population eating fortified vehicle vitamin a       NaN   
                          percent of population eating vehicle           na             True   
              wheat flour percent of population eating fortified vehicle folic acid      NaN   
                                                                         iron            NaN   
                                                                         zinc            NaN   
                          percent of population eating vehicle           na             True   
                                                                         na             True   

                                                                                                               inclusion_justification  \
location_name vehicle     value_description                              nutrient                                                        
Cameroon      oil         percent of population eating fortified vehicle vitamin a                                                 NaN   
                          percent of population eating vehicle           na          Keeping 53.1% for mixed/both urbanicity from E...   
              wheat flour percent of population eating fortified vehicle folic acid                                                NaN   
                                                                         iron                                                      NaN   
                                                                         zinc                                                      NaN   
                          percent of population eating vehicle           na          Keeping valu

## Cote d'Ivoire

In [41]:
civoire = check_one_country("Côte d'Ivoire")

#nathaniel marked these for inclusion
civoire.data_choice_notes = civoire.inclusion_justification

civoire[(civoire.vehicle.isin(vehicles))].groupby(['vehicle','value_description']).mean()

location_id
vehicle     value_description                                              
oil         percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN
wheat flour percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN

In [42]:
subset_data["Côte d'Ivoire"] = subset_data["Côte d'Ivoire"].append(
    civoire[(civoire.vehicle=="oil") & (civoire.included)]
)

civoire[(civoire.vehicle=="oil") & (civoire.included)]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,inclusion_justification,included,data_choice_notes
119,NaN,Côte d'Ivoire,mixed/both,na,oil,percent of population eating fortified vehicle,vitamin a,75,na,na,total population,2013,TABLE 2. Estimated coverage of vitamin A–forti...,"Sablah M, Klopp J, Steinberg D, Touaoro Z, Lai...",https://pubmed.ncbi.nlm.nih.gov/23444712/,Keeping Sablah et al. estimate of 75% from 201...,True,Keeping Sablah et al. estimate of 75% from 201...
825,NaN,Côte d'Ivoire,urban,Abidjan,oil,percent of population eating industrially prod...,na,98,97,99,households with children <2 years old,2014,"These are 95% CIs, rather than 97.5! Note the ...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source,True,Only one source
824,NaN,Côte d'Ivoire,urban,Abidjan,oil,percent of population eating vehicle,na,98.5,97.5,99.3,households with children <2 years old,2014,"These are 95% CIs, rather than 97.5! Note the ...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source,True,Only one source


In [43]:
subset_data["Côte d'Ivoire"] = subset_data["Côte d'Ivoire"].append(
    civoire[(civoire.vehicle=="wheat flour") & (civoire.included)]
)

civoire[(civoire.vehicle=="wheat flour") & (civoire.included)]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,inclusion_justification,included,data_choice_notes
823,NaN,Côte d'Ivoire,urban,Abidjan,wheat flour,percent of population eating industrially prod...,na,10.2,7.5,13.1,households with children <2 years old,2014,"These are 95% CIs, rather than 97.5! Note the ...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source,True,Only one source
822,NaN,Côte d'Ivoire,urban,Abidjan,wheat flour,percent of population eating vehicle,na,54.7,50.1,59.6,households with children <2 years old,2014,"These are 95% CIs, rather than 97.5! Note the ...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source,True,Only one source


In [44]:
subset_data["Côte d'Ivoire"] = subset_data["Côte d'Ivoire"].append(
    mult_estimates[(mult_estimates.location_name=="Côte d'Ivoire") 
               & (mult_estimates.vehicle=="wheat flour") 
               & (mult_estimates.nutrient.isin(nutrients))]
)

mult_estimates[(mult_estimates.location_name=="Côte d'Ivoire") 
               & (mult_estimates.vehicle=="wheat flour") 
               & (mult_estimates.nutrient.isin(nutrients))]

,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status
429,Côte d'Ivoire,wheat flour,folic acid,Mandatory,percent of population eating fortified vehicle,39.10698,multiplicative
430,Côte d'Ivoire,wheat flour,iron,Mandatory,percent of population eating fortified vehicle,24.49124,multiplicative
434,Côte d'Ivoire,wheat flour,zinc,Unknown,percent of population eating fortified vehicle,0.00000,multiplicative


In [45]:
mult_estimates[(mult_estimates.location_name=="Côte d'Ivoire") 
               & (mult_estimates.vehicle=="maize flour")]

,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status


In [46]:
# need all maize values from regression

## Democratic Republic of the Congo

In [47]:
drc = check_one_country("Democratic Republic of the Congo")

#nathaniel marked these for inclusion
drc.data_choice_notes = drc.inclusion_justification

drc[(drc.vehicle.isin(vehicles))].groupby(['vehicle','value_description']).mean()

,,location_id
vehicle,value_description,
maize flour,percent of population eating vehicle,NaN
oil,percent of population eating vehicle,NaN
wheat flour,percent of population eating vehicle,NaN


In [48]:
drc[(drc.vehicle=="maize flour")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,inclusion_justification,included,data_choice_notes
208,NaN,Democratic Republic of the Congo,rural,Turumbu,maize flour,percent of population eating vehicle,na,20.7,na,na,women,2009,"Described as ""village"" setting as opposed to ""...","Termote, C., Meyi, M. B., Djailo, B. D. A., Hu...",https://www-ncbi-nlm-nih-gov.offcampus.lib.was...,less representative than the moumin number (sa...,False,less representative than the moumin number (sa...
209,NaN,Democratic Republic of the Congo,urban,Turumbu,maize flour,percent of population eating vehicle,na,27.2,NaN,NaN,women,2009,"Described as ""village"" setting as opposed to ""...","Termote, C., Meyi, M. B., Djailo, B. D. A., Hu...",https://www-ncbi-nlm-nih-gov.offcampus.lib.was...,less representative than the moumin number (sa...,False,less representative than the moumin number (sa...
210,NaN,Democratic Republic of the Congo,urban,Kisangani,maize flour,percent of population eating vehicle,na,42.7,NaN,NaN,women,2009,"Asked about maize, not specifically maize flou...","Termote, C., Meyi, M. B., Djailo, B. D. A., Hu...",https://www-ncbi-nlm-nih-gov.offcampus.lib.was...,less representative than the moumin number (sa...,False,less representative than the moumin number (sa...
211,NaN,Democratic Republic of the Congo,mixed/both,Kongo Central,maize flour,percent of population eating vehicle,na,30,NaN,NaN,women of reproductive age,2014,women of reproductive age here = 15-49. Dietar...,"Moumin, N. A., Angel, M. D., Karakochuk, C. D....",https://www-ncbi-nlm-nih-gov.offcampus.lib.was...,wra instead of u5,False,wra instead of u5
212,NaN,Democratic Republic of the Congo,mixed/both,South Kivu,maize flour,percent of population eating vehicle,na,64,na,na,women of reproductive age,2014,women of reproductive age here = 15-49. Dietar...,"Moumin, N. A., Angel, M. D., Karakochuk, C. D....",https://www-ncbi-nlm-nih-gov.offcampus.lib.was...,wra instead of u5,False,wra instead of u5
213,NaN,Democratic Republic of the Congo,mixed/both,Kongo Central,maize flour,percent of population eating vehicle,na,31,NaN,NaN,under-5,2014,under-5 = 0.5-5 years. Dietary intake was asse...,"Moumin, N. A., Angel, M. D., Karakochuk, C. D....",https://www-ncbi-nlm-nih-gov.offcampus.lib.was...,"Discareded WRA data from same survey, in addit...",True,"Discareded WRA data from same survey, in addit..."
214,NaN,Democratic Republic of the Congo,mixed/both,South Kivu,maize flour,percent of population eating vehicle,na,53,na,na,under-5,2014,under-5 = 0.5-5 years. Dietary intake was asse...,"Moumin, N. A., Angel, M. D., Karakochuk, C. D....",https://www-ncbi-nlm-nih-gov.offcampus.lib.was...,"Discareded WRA data from same survey, in addit...",True,"Discareded WRA data from same survey, in addit..."
215,NaN,Democratic Republic of the Congo,rural,Bwamanda,maize flour,percent of population eating vehicle,na,93.44,90.19,95.7,under-5,1991,"The age group here is ""preschool"" the survey m...","Kismul, Hallgeir et al. “Diet and kwashiorkor:...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,"from 1991, small village",False,"from 1991, small village"


In [49]:
subset_data["Democratic Republic of the Congo"] = subset_data["Democratic Republic of the Congo"].append(
    drc[(drc.vehicle=="maize flour") & (drc.included)]
)

drc[(drc.vehicle=="maize flour") & (drc.included)]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,inclusion_justification,included,data_choice_notes
213,NaN,Democratic Republic of the Congo,mixed/both,Kongo Central,maize flour,percent of population eating vehicle,na,31,NaN,NaN,under-5,2014,under-5 = 0.5-5 years. Dietary intake was asse...,"Moumin, N. A., Angel, M. D., Karakochuk, C. D....",https://www-ncbi-nlm-nih-gov.offcampus.lib.was...,"Discareded WRA data from same survey, in addit...",True,"Discareded WRA data from same survey, in addit..."
214,NaN,Democratic Republic of the Congo,mixed/both,South Kivu,maize flour,percent of population eating vehicle,na,53,na,na,under-5,2014,under-5 = 0.5-5 years. Dietary intake was asse...,"Moumin, N. A., Angel, M. D., Karakochuk, C. D....",https://www-ncbi-nlm-nih-gov.offcampus.lib.was...,"Discareded WRA data from same survey, in addit...",True,"Discareded WRA data from same survey, in addit..."


In [50]:
subset_data["Democratic Republic of the Congo"] = subset_data["Democratic Republic of the Congo"].append(
    drc[(drc.vehicle=="oil") & (drc.included)]
)


drc.loc[(drc.vehicle=="oil") & (drc.included)]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,inclusion_justification,included,data_choice_notes
205,NaN,Democratic Republic of the Congo,rural,Turumbu,oil,percent of population eating vehicle,na,97.3,na,na,women,2009,"Described as ""village"" setting as opposed to ""...","Termote, C., Meyi, M. B., Djailo, B. D. A., Hu...",https://www-ncbi-nlm-nih-gov.offcampus.lib.was...,One of five subnational estimates for either u...,True,One of five subnational estimates for either u...
206,NaN,Democratic Republic of the Congo,urban,Turumbu,oil,percent of population eating vehicle,na,95.6,na,na,women,2009,"Described as ""village"" setting as opposed to ""...","Termote, C., Meyi, M. B., Djailo, B. D. A., Hu...",https://www-ncbi-nlm-nih-gov.offcampus.lib.was...,One of five subnational estimates for either u...,True,One of five subnational estimates for either u...
207,NaN,Democratic Republic of the Congo,urban,Kisangani,oil,percent of population eating vehicle,na,98.1,na,na,women,2009,"Asked about ""oil and fat"" = {vegetal oil + pal...","Termote, C., Meyi, M. B., Djailo, B. D. A., Hu...",https://www-ncbi-nlm-nih-gov.offcampus.lib.was...,One of five subnational estimates for either u...,True,One of five subnational estimates for either u...
217,NaN,Democratic Republic of the Congo,rural,Bwamanda,oil,percent of population eating vehicle,na,88.41,81.1,92.12,under-5,1991,"The age group here is ""preschool"" the survey m...","Kismul, Hallgeir et al. “Diet and kwashiorkor:...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,One of five subnational estimates for either u...,True,One of five subnational estimates for either u...
218,NaN,Democratic Republic of the Congo,rural,DRC Forest Concessions,oil,percent of population eating vehicle,na,100,na,na,total population,2012,This is specifally palm oil. The survey design...,"Donn, P., Nchuaji, T. E., Ngondi, J., Tieguhon...",https://www.researchgate.net/publication/28472...,One of five subnational estimates for either u...,True,One of five subnational estimates for either u...


In [51]:
subset_data["Democratic Republic of the Congo"] = subset_data["Democratic Republic of the Congo"].append(
    drc[(drc.vehicle=="wheat flour") & (drc.included)]
)

drc.loc[(drc.vehicle=="wheat flour") & (drc.included)]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,inclusion_justification,included,data_choice_notes
216,NaN,Democratic Republic of the Congo,rural,Bwamanda,wheat flour,percent of population eating vehicle,na,0.58,0,1.4,under-5,1991,"The age group here is ""preschool"" the survey m...","Kismul, Hallgeir et al. “Diet and kwashiorkor:...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,Only one source. Note: This data is from a 198...,True,Only one source. Note: This data is from a 198...


In [52]:
subset_data["Democratic Republic of the Congo"] = subset_data["Democratic Republic of the Congo"].append(
    mult_estimates[(mult_estimates.location_name=="Democratic Republic of the Congo") 
               & (mult_estimates.nutrient.isin(nutrients + ['NA']))].drop(columns='standard').drop_duplicates()
)

mult_estimates[(mult_estimates.location_name=="Democratic Republic of the Congo") 
               & (mult_estimates.nutrient.isin(nutrients + ['NA']))]

,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status
88,Democratic Republic of the Congo,maize flour,NA,Unknown,percent of population eating industrially prod...,2.26275,multiplicative
96,Democratic Republic of the Congo,wheat flour,NA,Voluntary,percent of population eating industrially prod...,0.58000,multiplicative
99,Democratic Republic of the Congo,wheat flour,NA,Unknown,percent of population eating industrially prod...,0.58000,multiplicative
439,Democratic Republic of the Congo,maize flour,folic acid,Unknown,percent of population eating fortified vehicle,0.00000,multiplicative
440,Democratic Republic of the Congo,maize flour,iron,Unknown,percent of population eating fortified vehicle,0.00000,multiplicative
444,Democratic Republic of the Congo,maize flour,zinc,Unknown,percent of population eating fortified vehicle,0.00000,multiplicative
445,Democratic Republic of the Congo,oil,vitamin a,Unknown,percent of population eating fortified vehicle,0.00000,multiplicative
447,Democratic Republic of the Congo,wheat flour,folic acid,Voluntary,percent of population eating fortified vehicle,0.07975,multiplicative
448,Democratic Republic of the Congo,wheat flour,iron,Voluntary,percent of population eating fortified vehicle,0.07975,multiplicative
452,Democratic Republic of the Congo,wheat flour,zinc,Unknown,percent of population eating fortified vehicle,0.07975,multiplicative


In [53]:
# need ind prod oil
# vitamin a fort wheat
# vitamin a fort maize

## Indonesia

In [54]:
indonesia = check_one_country("Indonesia")

#nathaniel marked these for inclusion
indonesia.data_choice_notes = indonesia.inclusion_justification

indonesia[(indonesia.vehicle.isin(vehicles))].groupby(['vehicle','value_description']).mean()

,,location_id
vehicle,value_description,
wheat flour,percent of population eating fortified vehicle,NaN


In [55]:
subset_data['Indonesia'] = subset_data['Indonesia'].append(
    indonesia[(indonesia.vehicle=="wheat flour") & (indonesia.nutrient.isin(nutrients)) & (indonesia.included)]
)

indonesia[(indonesia.vehicle=="wheat flour") & (indonesia.nutrient.isin(nutrients)) & (indonesia.included)]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,inclusion_justification,included,data_choice_notes
451,NaN,Indonesia,mixed/both,na,wheat flour,percent of population eating fortified vehicle,folic acid,73.1,na,na,total population,2020,I have calculated the percentage from the numb...,NaN,https://www.thejakartapost.com/academia/2020/0...,Only one source,True,Only one source
452,NaN,Indonesia,mixed/both,na,wheat flour,percent of population eating fortified vehicle,iron,73.1,na,na,total population,2020,I have calculated the percentage from the numb...,NaN,https://www.thejakartapost.com/academia/2020/0...,Only one source,True,Only one source
453,NaN,Indonesia,mixed/both,na,wheat flour,percent of population eating fortified vehicle,zinc,73.1,na,na,total population,2020,I have calculated the percentage from the numb...,NaN,https://www.thejakartapost.com/academia/2020/0...,Only one source,True,Only one source


In [56]:
mult_estimates[(mult_estimates.location_name=="Indonesia")]

,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status


In [57]:
# need all oil vars
# need pct eating wheat and pct eating ind prod wheat

## Mozambique

In [58]:
moz = check_one_country("Mozambique")

#nathaniel marked these for inclusion
moz.data_choice_notes = moz.inclusion_justification

moz[(moz.vehicle.isin(vehicles))].groupby(['vehicle','value_description']).mean()

,,location_id
vehicle,value_description,
maize flour,percent of population eating vehicle,NaN
oil,percent of population eating vehicle,NaN
wheat flour,percent of population eating vehicle,NaN


In [59]:
subset_data['Mozambique'] = subset_data['Mozambique'].append(
    moz[(moz.vehicle=="maize flour") & (moz.included)]
) 

moz[(moz.vehicle=="maize flour") & (moz.included)]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,inclusion_justification,included,data_choice_notes
582,NaN,Mozambique,mixed/both,na,maize flour,percent of population eating vehicle,na,79.9,na,na,total population,2013,Population coverage of food. Ministry of Healt...,GFDX,https://fortificationdata.org/country-fortific...,Only one source.,True,Only one source.


In [60]:
subset_data['Mozambique'] = subset_data['Mozambique'].append(
    moz[(moz.vehicle=="oil") & (moz.included)]
) 

moz[(moz.vehicle=="oil") & (moz.included)]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,inclusion_justification,included,data_choice_notes
793,NaN,Mozambique,mixed/both,na,oil,percent of population eating vehicle,na,95.2,na,na,total population,2013,Population coverage of food. Ministry of Healt...,GFDX,https://fortificationdata.org/country-fortific...,Keeping GFDx value (95.2%) for total populatio...,True,Keeping GFDx value (95.2%) for total populatio...


In [61]:
subset_data['Mozambique'] = subset_data['Mozambique'].append(
    moz[(moz.vehicle=="wheat flour") & (moz.included)]
) 

moz[(moz.vehicle=="wheat flour") & (moz.included)]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,inclusion_justification,included,data_choice_notes
797,NaN,Mozambique,mixed/both,na,wheat flour,percent of population eating vehicle,na,16.3,na,na,total population,2013,Population coverage of food. Ministry of Healt...,GFDX,https://fortificationdata.org/country-fortific...,"Keeping GFDx estimate (16.3%), as it is more r...",True,"Keeping GFDx estimate (16.3%), as it is more r..."


In [62]:
mult_estimates[(mult_estimates.location_name=="Mozambique")]

,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status
206,Mozambique,maize flour,NA,Mandatory,percent of population eating industrially prod...,23.970000,multiplicative
214,Mozambique,wheat flour,NA,Mandatory,percent of population eating industrially prod...,66.766667,multiplicative
557,Mozambique,maize flour,folic acid,Mandatory,percent of population eating fortified vehicle,36.354500,multiplicative
558,Mozambique,maize flour,iron,Mandatory,percent of population eating fortified vehicle,36.354500,multiplicative
559,Mozambique,maize flour,vitamin a,Mandatory,percent of population eating fortified vehicle,36.354500,multiplicative
560,Mozambique,maize flour,vitamin b1,Mandatory,percent of population eating fortified vehicle,36.354500,multiplicative
561,Mozambique,maize flour,vitamin b12,Mandatory,percent of population eating fortified vehicle,36.354500,multiplicative
562,Mozambique,maize flour,zinc,Mandatory,percent of population eating fortified vehicle,36.354500,multiplicative
563,Mozambique,oil,vitamin a,Mandatory,percent of population eating fortified vehicle,37.577400,multiplicative
565,Mozambique,wheat flour,folic acid,Mandatory,percent of population eating fortified vehicle,40.060000,multiplicative


In [63]:
subset_data['Mozambique'] = subset_data['Mozambique'].append(
    mult_estimates[(mult_estimates.location_name=="Mozambique") & (mult_estimates.nutrient.isin(["NA"] + nutrients))]
)

mult_estimates[(mult_estimates.location_name=="Mozambique") & (mult_estimates.nutrient.isin(["NA"] + nutrients))]

,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status
206,Mozambique,maize flour,NA,Mandatory,percent of population eating industrially prod...,23.970000,multiplicative
214,Mozambique,wheat flour,NA,Mandatory,percent of population eating industrially prod...,66.766667,multiplicative
557,Mozambique,maize flour,folic acid,Mandatory,percent of population eating fortified vehicle,36.354500,multiplicative
558,Mozambique,maize flour,iron,Mandatory,percent of population eating fortified vehicle,36.354500,multiplicative
559,Mozambique,maize flour,vitamin a,Mandatory,percent of population eating fortified vehicle,36.354500,multiplicative
562,Mozambique,maize flour,zinc,Mandatory,percent of population eating fortified vehicle,36.354500,multiplicative
563,Mozambique,oil,vitamin a,Mandatory,percent of population eating fortified vehicle,37.577400,multiplicative
565,Mozambique,wheat flour,folic acid,Mandatory,percent of population eating fortified vehicle,40.060000,multiplicative
566,Mozambique,wheat flour,iron,Mandatory,percent of population eating fortified vehicle,51.744167,multiplicative
567,Mozambique,wheat flour,vitamin a,Mandatory,percent of population eating fortified vehicle,40.060000,multiplicative


In [64]:
## need pct eating ind prod oil

## pop-weight subnationals

In [65]:
#these are the subnats we have to weight

checkout = pd.concat(list(subset_data.values()))
checkout.loc[(checkout.subnational_name.notna()) & (checkout.subnational_name!='na'),
         ['location_name','urbanicity','subnational_name','source_link']].drop_duplicates()

/ihme/homes/beatrixh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,location_name,urbanicity,subnational_name,source_link
825,Côte d'Ivoire,urban,Abidjan,https://doi.org/10.3945/jn.116.245753
213,Democratic Republic of the Congo,mixed/both,Kongo Central,https://www-ncbi-nlm-nih-gov.offcampus.lib.was...
214,Democratic Republic of the Congo,mixed/both,South Kivu,https://www-ncbi-nlm-nih-gov.offcampus.lib.was...
205,Democratic Republic of the Congo,rural,Turumbu,https://www-ncbi-nlm-nih-gov.offcampus.lib.was...
206,Democratic Republic of the Congo,urban,Turumbu,https://www-ncbi-nlm-nih-gov.offcampus.lib.was...
207,Democratic Republic of the Congo,urban,Kisangani,https://www-ncbi-nlm-nih-gov.offcampus.lib.was...
217,Democratic Republic of the Congo,rural,Bwamanda,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...
218,Democratic Republic of the Congo,rural,DRC Forest Concessions,https://www.researchgate.net/publication/28472...


In [66]:
# going to urbanicity-weight the DRC ones

In [67]:
tmp = subset_data['Democratic Republic of the Congo']

In [68]:
tmp[['vehicle','value_description','nutrient','urbanicity','value_mean','value_025_percentile','value_975_percentile']]

,vehicle,value_description,nutrient,urbanicity,value_mean,value_025_percentile,value_975_percentile
213,maize flour,percent of population eating vehicle,na,mixed/both,31,NaN,NaN
214,maize flour,percent of population eating vehicle,na,mixed/both,53,na,na
205,oil,percent of population eating vehicle,na,rural,97.3,na,na
206,oil,percent of population eating vehicle,na,urban,95.6,na,na
207,oil,percent of population eating vehicle,na,urban,98.1,na,na
217,oil,percent of population eating vehicle,na,rural,88.41,81.1,92.12
218,oil,percent of population eating vehicle,na,rural,100,na,na
216,wheat flour,percent of population eating vehicle,na,rural,0.58,0,1.4
88,maize flour,percent of population eating industrially prod...,NA,NaN,2.26275,NaN,NaN
96,wheat flour,percent of population eating industrially prod...,NA,NaN,0.58,NaN,NaN


In [69]:
tmp.loc[(tmp.value_mean=='na'),'value_mean'] = np.nan
tmp.loc[(tmp.value_025_percentile=='na'),'value_025_percentile'] = np.nan
tmp.loc[(tmp.value_975_percentile=='na'),'value_975_percentile'] = np.nan

tmp.value_mean = tmp.value_mean.astype(float)
tmp.value_025_percentile = tmp.value_025_percentile.astype(float)
tmp.value_975_percentile = tmp.value_975_percentile.astype(float)

In [70]:
tmp.loc[(tmp.nutrient.isna()),'nutrient'] = 'na'
tmp.loc[(tmp.urbanicity.isna()),'urbanicity'] = 'na'

In [71]:
## south kivu population 

In [72]:
tmp_maize = tmp[(tmp.vehicle=="maize flour") & (tmp.value_description=="percent of population eating vehicle")]
tmp = tmp[~((tmp.vehicle=="maize flour") & (tmp.value_description=="percent of population eating vehicle"))]

pop_drc = {
    'South Kivu':5_772_000,
    'Kongo Central':5_575_000
}

sum(pop_drc.values())
tmp_maize['pop_weight']  = tmp_maize.subnational_name.map(pop_drc)
tmp_maize['pop_weight']  = tmp_maize.pop_weight / sum(pop_drc.values())

tmp_maize['value_mean'] = tmp_maize.value_mean * tmp_maize.pop_weight
tmp_maize['value_mean'] = tmp_maize.value_mean.sum()

tmp_maize = tmp_maize.drop(columns=['subnational_name','pop_weight'])
tmp_maize = tmp_maize.drop_duplicates()

/ihme/homes/beatrixh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/ihme/homes/beatrixh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/ihme/homes/beatrixh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [73]:
assert(len(tmp_maize)==1)
tmp = tmp.append(tmp_maize)

In [74]:
tmp_oil = tmp[(tmp.vehicle=="oil") & (tmp.value_description=="percent of population eating vehicle")]
tmp = tmp[~((tmp.vehicle=="oil") & (tmp.value_description=="percent of population eating vehicle"))]

In [75]:
tmp_oil['value_mean'] = tmp_oil.value_mean.mean()

## this makes no sense, also these are all nans
tmp_oil['value_025_percentile'] = tmp_oil.value_025_percentile.min()
tmp_oil['value_975_percentile'] = tmp_oil.value_975_percentile.max()

In [76]:
tmp_oil['source_citation'] = ', '.join(tmp_oil.source_citation.unique().tolist())
tmp_oil['source_link'] = ', '.join(tmp_oil.source_link.unique().tolist())

In [77]:
tmp_oil = tmp_oil.drop(columns=['notes','sub_population','source_year','subnational_name','urbanicity']).drop_duplicates()

In [78]:
tmp_oil['data_choice_notes'] = 'Took mean of five different subnational estimates from different years for u5, total population, or women'

In [79]:
assert(len(tmp_oil)==1)
tmp = tmp.append(tmp_oil)

In [80]:
subset_data['Democratic Republic of the Congo'] = tmp

## Check for missingness

In [81]:
all_data = pd.concat(list(subset_data.values()))

/ihme/homes/beatrixh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [82]:
all_data.data_choice_notes.unique()

array(['Keeping 53.1% for mixed/both urbanicity from Engle-Stone et al. 2012. Discarding values for urban populations (81.2%, 91.7%) from Engle-Stone et al. 2017. Confidence intervals missing, but same source reports percent of WRA consuming oil is 54.1% (48.5%, 59.7%)',
       nan,
       'Keeping value from Hess et al. (92%) for mixed/both urbanicity from 2011 FRAT survey. Value from Engle-Stone et al. 2012 from 2009 survey was similar (92.2%). Discarding values for urban populations (98%, 97.6%, 95.2%) from Engle-Stone et al. 2012 and Engle-Stone et al. 2016, and for Southern and Northern Ecological Zones (91.2%, 89.8%) from Engle-Stone et al. 2012. ',
       'Keeping value from Hess et al. (94%) from 2011 FRAT survey for percent of population eating wheat flour. Discarding older value (93.7%) with no uncertainty from Engle-Stone et al. 2009. Discarding estimates for urban population (98.4% in 2012, 97% in 2009) from Engle-Stone et al. 2016.',
       'Keeping Sablah et al. estimate 

In [83]:
all_data.loc[~(all_data.nutrient.isin(['vitamin a','iron','zinc','folic acid'])),'nutrient'] = 'na'

In [84]:
all_data[rcols + ['value_mean','value_025_percentile','value_975_percentile','sub_population']]

,location_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population
610,Cameroon,oil,percent of population eating vehicle,na,53.1,na,na,under-5
401,Cameroon,oil,percent of population eating fortified vehicle,vitamin a,30.4969,NaN,NaN,NaN
575,Cameroon,wheat flour,percent of population eating vehicle,na,92,90,94,women of reproductive age
577,Cameroon,wheat flour,percent of population eating vehicle,na,94,92,96,under-5
403,Cameroon,wheat flour,percent of population eating fortified vehicle,folic acid,78.2159,NaN,NaN,NaN
404,Cameroon,wheat flour,percent of population eating fortified vehicle,iron,59.7246,NaN,NaN,NaN
408,Cameroon,wheat flour,percent of population eating fortified vehicle,zinc,78.2159,NaN,NaN,NaN
119,Côte d'Ivoire,oil,percent of population eating fortified vehicle,vitamin a,75,na,na,total population
825,Côte d'Ivoire,oil,percent of population eating industrially prod...,na,98,97,99,households with children <2 years old
824,Côte d'Ivoire,oil,percent of population eating vehicle,na,98.5,97.5,99.3,households with children <2 years old


In [85]:
check = target.merge(all_data[rcols + ['value_mean']], on = rcols, how = 'left')

In [88]:
check[check.value_mean.isna()].sort_values(['value_description','vehicle'])

,location_name,vehicle,value_description,nutrient,value_mean
0,Cameroon,maize flour,percent of population eating fortified vehicle,folic acid,NaN
1,Cameroon,maize flour,percent of population eating fortified vehicle,iron,NaN
2,Cameroon,maize flour,percent of population eating fortified vehicle,vitamin a,NaN
3,Cameroon,maize flour,percent of population eating fortified vehicle,zinc,NaN
16,Côte d'Ivoire,maize flour,percent of population eating fortified vehicle,folic acid,NaN
17,Côte d'Ivoire,maize flour,percent of population eating fortified vehicle,iron,NaN
18,Côte d'Ivoire,maize flour,percent of population eating fortified vehicle,vitamin a,NaN
19,Côte d'Ivoire,maize flour,percent of population eating fortified vehicle,zinc,NaN
33,Democratic Republic of the Congo,maize flour,percent of population eating fortified vehicle,vitamin a,NaN
46,Indonesia,oil,percent of population eating fortified vehicle,vitamin a,NaN


In [ ]:
assert(len(check[check.value_mean.isna()])==0), "there are target loc/vehicle/val/nutrient combos youre missing"

In [ ]:
rcols

In [ ]:
output = all_data[rcols + ['value_mean','value_025_percentile', 'value_975_percentile','sub_population','estimation_status','source_citation','source_link','data_choice_notes']].sort_values(rcols).set_index(rcols)

In [ ]:
##impute all missing CIs

# clean value_mean
output.loc[output.value_mean=='na','value_mean'] = np.nan
output.value_mean = output.value_mean.astype(float)

# clean 2.5th %ile
output.loc[output.value_025_percentile=='na','value_025_percentile'] = np.nan
output.value_025_percentile = output.value_025_percentile.astype(float)

# clean 97.5th %ile
output.loc[output.value_975_percentile=='na','value_975_percentile'] = np.nan
output.value_975_percentile = output.value_975_percentile.astype(float)

In [ ]:
output.loc[(output.value_mean > output.value_975_percentile),'value_975_percentile'] = np.nan

In [ ]:
output.loc[(output.value_mean < output.value_025_percentile)]

In [ ]:
output.loc[(output.value_mean == output.value_025_percentile)]

In [ ]:
output['scale_over_mean'] = (output.value_975_percentile - output.value_025_percentile) / output.value_mean

In [ ]:
output = output.reset_index()

In [ ]:
r = output.loc[(output.scale_over_mean!=np.inf),['vehicle','scale_over_mean']]
# .groupby('vehicle').mean().dropna().rename(columns={'scale_over_mean':'r'}).reset_index()

In [ ]:
r_mean = r.scale_over_mean.mean()

In [ ]:
r.loc[r.vehicle=="maize flour",'scale_over_mean'] = r_mean

In [ ]:
r = r.groupby('vehicle').mean().dropna().rename(columns={'scale_over_mean':'r'}).reset_index()

In [ ]:
# add uncertainty
output = output.merge(r, on = 'vehicle', how = 'outer')
output['lower'] = np.clip(output.value_mean - (output.r * output.value_mean)/2, 0, 100)
output['upper'] = np.clip(output.value_mean + (output.r * output.value_mean)/2, 0, 100)

In [ ]:
output

In [ ]:
output.loc[(output.value_mean < output.value_025_percentile) | (output.value_025_percentile.isna()),'CI_source'] = "modeling"
output.loc[(output.value_mean > output.value_975_percentile) | (output.value_975_percentile.isna()),'CI_source'] = "modeling"

output.loc[output.CI_source.isna(),'CI_source'] = 'extraction'

output.loc[(output.value_mean < output.value_025_percentile) | (output.value_025_percentile.isna()),'value_025_percentile'] = output.loc[(output.value_mean < output.value_025_percentile) | (output.value_025_percentile.isna())].lower
output.loc[(output.value_mean > output.value_975_percentile) | (output.value_975_percentile.isna()),'value_975_percentile'] = output.loc[(output.value_mean > output.value_975_percentile) | (output.value_975_percentile.isna())].upper

output = output.drop(columns=['r','lower','upper','scale_over_mean']).set_index(rcols)

output.loc[output.estimation_status.isna(),'estimation_status'] = 'na'

In [ ]:
output

In [ ]:
output = output.reset_index()

In [ ]:
output.loc[(output.location_name=="Vietnam"),'location_name'] = "Viet Nam"

In [ ]:
sort_helper = {
    'percent of population eating fortified vehicle': 'C',
 'percent of population eating industrially produced vehicle': 'B',
 'percent of population eating vehicle': 'A'}

In [ ]:
output['sort_helper'] = output.value_description.map(sort_helper)

In [ ]:
output = output.sort_values(['location_name','vehicle','sort_helper','value_description','nutrient']).drop(columns='sort_helper')

In [ ]:
save_path_tmp = '/ihme/homes/beatrixh/repos/scratch/tier3_coverage_data_03_23_2021.csv'
output.to_csv(save_path_tmp, index = False)

In [ ]:
break

In [ ]:
save_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/population_coverage_data_tier3_locs_3_23_2021.csv'
output.to_csv(save_path, index = False)

In [ ]:
test[(tset.)]

In [ ]:
test = output.reset_index().copy()

In [ ]:
test[(test.location_name=="Myanmar")]

In [ ]:
test[(test.vehicle=="oil")].groupby(['location_name','vehicle','value_description','nutrient']).mean()

In [ ]:
test[(test.vehicle=="wheat flour")].groupby(['location_name','vehicle','value_description']).mean()

In [ ]:
test[(test.vehicle=="maize flour")].groupby(['location_name','vehicle','value_description']).mean()

In [ ]:
test.columns

In [ ]:
pd.pivot_table(values='value_mean', index = [''])

In [ ]:
## TODO 
# check whats missing
# pop weight as necessary
# rerun the regressions --- make sure to fix burkina faso
# incorporate and format 